# 实验 4.3：差分隐私防御实战

## 学习目标
完成本实验后，你将能够：
- 解释差分隐私（DP）的核心机制：梯度裁剪与噪声添加
- 实现简化版的差分隐私 SGD（DP-SGD）训练流程
- 对比普通训练与 DP 训练在模型性能上的差异
- 评估差分隐私对成员推理攻击的防御效果
- 分析隐私预算（ε）与模型效用之间的权衡关系

## 前置知识
- 理解梯度下降和反向传播的基本原理
- 了解成员推理攻击的威胁模型（完成实验 4.2）
- 熟悉 PyTorch 基础操作（张量、模型训练）

## 实验环境
- 平台：本地环境或腾讯 Cloud Studio
- GPU：可选（CPU 即可完成）
- 模型：自定义 PyTorch 分类器
- 数据集：合成多分类数据集

## 填空说明
本实验共 **5 个填空**，难度：⭐⭐⭐☆☆
- 填空 1-2：实现差分隐私核心机制（梯度裁剪、噪声添加）
- 填空 3-4：模型评估与防御效果分析
- 填空 5：隐私-效用权衡实验

⚠️ **重要提示**：差分隐私是理论严谨的隐私保护技术，本实验为简化教学版本，实际应用请使用专业库（如 Opacus）。

## 第一部分：环境准备

### 1.1 安装依赖

In [ ]:
# 安装必要的库
!pip install torch numpy matplotlib scikit-learn -q

print("✓ 依赖安装完成！")

In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子，确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("="*60)
print("🔧 环境配置")
print("="*60)
print(f"  PyTorch 版本: {torch.__version__}")
print(f"  设备: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print("  ✓ 环境准备完成！")

### 1.2 创建实验数据集

In [ ]:
# 生成合成分类数据集
def create_dataset(n_samples=1000, n_features=20, n_classes=5, test_size=0.5, random_state=42):
    """
    生成用于训练和测试的分类数据集
    
    参数:
        n_samples: 总样本数
        n_features: 特征维度
        n_classes: 类别数
        test_size: 测试集比例
    
    返回:
        X_train, y_train, X_test, y_test (张量格式)
    """
    # 生成多分类数据
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=15,
        n_redundant=5,
        n_classes=n_classes,
        random_state=random_state
    )
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    # 转换为 PyTorch 张量
    return (
        torch.FloatTensor(X_train),
        torch.LongTensor(y_train),
        torch.FloatTensor(X_test),
        torch.LongTensor(y_test)
    )

# 创建数据集
X_train, y_train, X_test, y_test = create_dataset()

print("="*60)
print("📊 数据集信息")
print("="*60)
print(f"  训练集: {len(X_train)} 样本")
print(f"  测试集: {len(X_test)} 样本")
print(f"  特征维度: {X_train.shape[1]}")
print(f"  类别数: {len(torch.unique(y_train))}")
print("  ✓ 数据集创建完成！")

### 1.3 定义模型架构

In [ ]:
# 定义简单的前馈神经网络分类器
class SimpleClassifier(nn.Module):
    """
    简单的三层全连接神经网络
    
    结构: 输入层 -> 隐藏层1(64) -> 隐藏层2(64) -> 输出层
    """
    def __init__(self, n_features=20, n_classes=5, hidden_size=64):
        super(SimpleClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, n_classes)
        )
    
    def forward(self, x):
        return self.network(x)

# 测试模型
test_model = SimpleClassifier()
test_input = torch.randn(4, 20)
test_output = test_model(test_input)

print("="*60)
print("🧠 模型架构")
print("="*60)
print(test_model)
print(f"\n  测试输入形状: {test_input.shape}")
print(f"  测试输出形状: {test_output.shape}")
print("  ✓ 模型定义完成！")

运行到这里，你应该看到：
- 环境配置成功，显示 PyTorch 版本
- 数据集包含 500 个训练样本和 500 个测试样本
- 模型架构清晰显示，包含 3 层全连接网络

如果遇到问题，请检查是否成功安装了 PyTorch 和相关依赖。

## 第二部分：差分隐私核心机制

💡 **什么是差分隐私？**

**差分隐私（Differential Privacy, DP）** 是一种严格的隐私保护标准，它确保：即使攻击者拥有除一条记录外的所有数据，也无法确定该记录是否在训练集中。

### 核心思想

通过在训练过程中添加**精心设计的噪声**，使得单个样本的存在对模型输出的影响可以被忽略。

### DP-SGD 的两个关键步骤

```
普通训练:  样本 → 计算梯度 → 累加 → 更新参数
                      ↓
DP训练:    样本 → 计算梯度 → 裁剪 → 添加噪声 → 更新参数
                         [控制影响] [保护隐私]
```

1. **梯度裁剪（Gradient Clipping）**：限制每个样本的梯度范数不超过阈值 C
   - 作用：防止某个样本的梯度过大，主导模型更新
   - 公式：`g' = g * min(1, C / ||g||)`

2. **噪声添加（Noise Addition）**：在聚合梯度上添加高斯噪声
   - 作用：隐藏单个样本对梯度的具体贡献
   - 公式：`g_final = g_avg + N(0, σ²C²)`

### 隐私预算 ε（Epsilon）

| ε 值 | 隐私保护 | 噪声大小 | 模型性能 |
|------|----------|----------|----------|
| 小（0.1-1） | 强 | 大 | 较差 |
| 中（1-3） | 中等 | 中等 | 平衡 |
| 大（3-10） | 弱 | 小 | 较好 |

### 2.1 实现差分隐私的核心函数

In [ ]:
# ========== 填空 1：实现梯度裁剪函数 ==========
# 
# 🎯 任务：实现梯度裁剪，限制梯度的 L2 范数不超过 max_norm
# 
# 💡 提示：
#   - 先计算梯度的 L2 范数：torch.norm(gradient)
#   - 如果范数 > max_norm，则按比例缩放：gradient * (max_norm / 当前范数)
#   - 如果范数 ≤ max_norm，则不变
#
# 难度：⭐⭐☆☆☆

def clip_gradient(gradient, max_norm):
    """
    裁剪梯度，确保其 L2 范数不超过 max_norm
    
    参数:
        gradient: 梯度张量
        max_norm: 最大范数阈值（裁剪阈值 C）
    
    返回:
        裁剪后的梯度
    """
    # 步骤1：计算梯度的 L2 范数
    grad_norm = torch.norm(gradient)
    
    # 步骤2：如果范数超过阈值，进行裁剪
    # 请将 ___________ 替换为正确的裁剪公式
    if grad_norm > max_norm:
        gradient = ___________  # 期望：将梯度范数缩放到 max_norm
    
    return gradient

# 测试梯度裁剪
print("="*60)
print("🧪 测试梯度裁剪功能")
print("="*60)

test_grad = torch.randn(10) * 5  # 生成一个大梯度
original_norm = torch.norm(test_grad).item()
clipped_grad = clip_gradient(test_grad.clone(), max_norm=1.0)
clipped_norm = torch.norm(clipped_grad).item()

print(f"  原始梯度范数: {original_norm:.4f}")
print(f"  裁剪后范数: {clipped_norm:.4f}")
print(f"  裁剪阈值: 1.0")

if clipped_norm <= 1.001:  # 允许微小的数值误差
    print("  ✓ 梯度裁剪功能正常！")
else:
    print("  ✗ 梯度裁剪可能有误，请检查填空 1")

In [ ]:
# ========== 填空 2：实现噪声添加机制 ==========
# 
# 🎯 任务：在平均梯度上添加高斯噪声，实现差分隐私保护
# 
# 💡 提示：
#   - 噪声标准差计算公式：noise_std = (max_grad_norm * noise_multiplier) / n_samples
#   - 使用 torch.randn_like(tensor) 生成与 tensor 同形状的标准正态分布随机数
#   - 将随机数乘以 noise_std 得到最终噪声
#
# 难度：⭐⭐⭐☆☆

def add_noise_to_gradient(avg_gradient, max_grad_norm, noise_multiplier, n_samples):
    """
    在平均梯度上添加高斯噪声
    
    参数:
        avg_gradient: 平均梯度
        max_grad_norm: 梯度裁剪阈值 C
        noise_multiplier: 噪声乘数 σ（控制隐私强度）
        n_samples: 样本数量
    
    返回:
        添加噪声后的梯度
    """
    # 步骤1：计算噪声标准差
    noise_std = (max_grad_norm * noise_multiplier) / n_samples
    
    # 步骤2：生成高斯噪声
    # 请将 ___________ 替换为生成噪声的代码
    noise = ___________  # 期望：生成 N(0, noise_std²) 的噪声
    
    # 步骤3：将噪声添加到梯度上
    noisy_gradient = avg_gradient + noise
    
    return noisy_gradient

# 测试噪声添加
print("="*60)
print("🧪 测试噪声添加功能")
print("="*60)

test_avg_grad = torch.ones(10) * 0.1
noisy_grad = add_noise_to_gradient(test_avg_grad.clone(), max_grad_norm=1.0, 
                                   noise_multiplier=1.0, n_samples=100)

noise_magnitude = torch.norm(noisy_grad - test_avg_grad).item()
print(f"  原始梯度均值: {test_avg_grad.mean().item():.4f}")
print(f"  添加噪声后均值: {noisy_grad.mean().item():.4f}")
print(f"  噪声幅度: {noise_magnitude:.4f}")

if noise_magnitude > 0.001:
    print("  ✓ 噪声添加功能正常！")
else:
    print("  ✗ 噪声可能未正确添加，请检查填空 2")

### 🤔 思考一下

1. **为什么需要先裁剪梯度再添加噪声？** 如果不裁剪直接添加噪声会有什么问题？

2. **噪声标准差为什么除以样本数 n_samples？** 这与隐私保护有什么关系？

## 第三部分：实现差分隐私训练

### 3.1 实现完整的 DP-SGD 训练流程

In [ ]:
def train_with_dp(model, X_train, y_train, epochs=50, lr=0.01, 
                  max_grad_norm=1.0, noise_multiplier=1.0, verbose=True):
    """
    使用差分隐私训练模型（简化版 DP-SGD）
    
    参数:
        model: PyTorch 模型
        X_train, y_train: 训练数据
        epochs: 训练轮数
        lr: 学习率
        max_grad_norm: 梯度裁剪阈值 C
        noise_multiplier: 噪声乘数 σ
        verbose: 是否打印训练过程
    
    返回:
        losses: 每个 epoch 的损失值列表
    """
    criterion = nn.CrossEntropyLoss()
    losses = []
    accuracies = []
    n_samples = len(X_train)
    
    if verbose:
        print(f"\n🔐 开始差分隐私训练（噪声强度={noise_multiplier}）")
        print(f"  梯度裁剪阈值: {max_grad_norm}")
        print(f"  训练样本数: {n_samples}")
        print(f"  学习率: {lr}\n")
    
    for epoch in range(epochs):
        model.train()
        
        # 初始化累积梯度字典
        accumulated_grads = {name: torch.zeros_like(param) 
                            for name, param in model.named_parameters()}
        
        # 对每个样本单独计算梯度（per-sample gradient）
        for i in range(n_samples):
            # 前向传播
            output = model(X_train[i:i+1])
            loss = criterion(output, y_train[i:i+1])
            
            # 反向传播计算梯度
            model.zero_grad()
            loss.backward()
            
            # 对每个参数的梯度进行裁剪并累加
            for name, param in model.named_parameters():
                if param.grad is not None:
                    # 裁剪单个样本的梯度
                    clipped_grad = clip_gradient(param.grad.clone(), max_grad_norm)
                    # 累加到总梯度
                    accumulated_grads[name] += clipped_grad
        
        # 计算平均梯度并添加噪声
        for name, param in model.named_parameters():
            # 计算平均梯度
            avg_grad = accumulated_grads[name] / n_samples
            
            # 添加高斯噪声
            noisy_grad = add_noise_to_gradient(
                avg_grad, max_grad_norm, noise_multiplier, n_samples
            )
            
            # 使用带噪声的梯度更新参数
            param.data -= lr * noisy_grad
        
        # 计算当前 epoch 的损失和准确率
        model.eval()
        with torch.no_grad():
            outputs = model(X_train)
            epoch_loss = criterion(outputs, y_train).item()
            epoch_acc = (outputs.argmax(dim=1) == y_train).float().mean().item()
            losses.append(epoch_loss)
            accuracies.append(epoch_acc)
        
        # 打印训练进度
        if verbose and (epoch + 1) % 10 == 0:
            print(f"  Epoch [{epoch+1:2d}/{epochs}] - Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2%}")
    
    if verbose:
        print("  ✓ 训练完成！\n")
    
    return losses, accuracies

print("✓ 差分隐私训练函数定义完成！")

### 3.2 实现普通训练函数（用于对比）

In [ ]:
def train_normal(model, X_train, y_train, epochs=50, lr=0.01, verbose=True):
    """
    普通训练（无隐私保护）
    
    参数:
        model: PyTorch 模型
        X_train, y_train: 训练数据
        epochs: 训练轮数
        lr: 学习率
        verbose: 是否打印训练过程
    
    返回:
        losses: 每个 epoch 的损失值列表
    """
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    losses = []
    accuracies = []
    
    if verbose:
        print(f"\n🔓 开始普通训练（无隐私保护）")
        print(f"  学习率: {lr}\n")
    
    for epoch in range(epochs):
        model.train()
        
        # 标准的批量梯度下降
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        # 记录损失和准确率
        losses.append(loss.item())
        acc = (outputs.argmax(dim=1) == y_train).float().mean().item()
        accuracies.append(acc)
        
        # 打印训练进度
        if verbose and (epoch + 1) % 10 == 0:
            print(f"  Epoch [{epoch+1:2d}/{epochs}] - Loss: {loss.item():.4f} | Acc: {acc:.2%}")
    
    if verbose:
        print("  ✓ 训练完成！\n")
    
    return losses, accuracies

print("✓ 普通训练函数定义完成！")

### 3.3 训练两个模型进行对比

In [ ]:
print("="*60)
print("🚀 开始训练模型")
print("="*60)

# 训练普通模型
normal_model = SimpleClassifier()
normal_losses, normal_accs = train_normal(normal_model, X_train, y_train, epochs=50, lr=0.01)

# 训练差分隐私模型
dp_model = SimpleClassifier()
dp_losses, dp_accs = train_with_dp(dp_model, X_train, y_train, epochs=50, lr=0.01,
                                    max_grad_norm=1.0, noise_multiplier=1.0)

### 3.4 可视化训练过程

In [ ]:
# 绘制训练损失和准确率对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 损失曲线
axes[0].plot(normal_losses, label='普通训练', color='steelblue', linewidth=2)
axes[0].plot(dp_losses, label='差分隐私训练 (σ=1.0)', color='coral', linewidth=2)
axes[0].set_xlabel('训练轮数 (Epoch)', fontsize=12)
axes[0].set_ylabel('损失值 (Loss)', fontsize=12)
axes[0].set_title('📉 训练损失对比', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# 准确率曲线
axes[1].plot(normal_accs, label='普通训练', color='steelblue', linewidth=2)
axes[1].plot(dp_accs, label='差分隐私训练 (σ=1.0)', color='coral', linewidth=2)
axes[1].set_xlabel('训练轮数 (Epoch)', fontsize=12)
axes[1].set_ylabel('准确率 (Accuracy)', fontsize=12)
axes[1].set_title('📈 训练准确率对比', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 观察：")
print("  - 普通模型收敛更快，损失下降更明显")
print("  - DP 模型由于噪声影响，训练曲线更加波动")
print("  - 这是隐私保护的代价：噪声干扰了正常的梯度下降")

运行到这里，你应该看到：
- 两个模型都成功完成 50 轮训练
- 普通模型的训练准确率通常在 85% 以上
- DP 模型的训练准确率略低于普通模型（约 75-85%）
- DP 模型的损失曲线更加波动（噪声导致）

如果 DP 模型准确率过低（<60%），可以尝试减小 noise_multiplier 或增加训练轮数。

## 第四部分：评估模型性能与隐私保护

### 4.1 评估模型在测试集上的泛化性能

In [ ]:
# ========== 填空 3：实现模型评估函数 ==========
# 
# 🎯 任务：计算模型在给定数据集上的准确率
# 
# 💡 提示：
#   - 使用 model.eval() 切换到评估模式
#   - 使用 torch.no_grad() 禁用梯度计算
#   - outputs.argmax(dim=1) 获取预测类别
#   - (pred == y).float().mean() 计算准确率
#
# 难度：⭐⭐☆☆☆

def evaluate_model(model, X, y):
    """
    评估模型在给定数据集上的准确率
    
    参数:
        model: PyTorch 模型
        X, y: 评估数据
    
    返回:
        accuracy: 准确率（0-1 之间的浮点数）
    """
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        predictions = outputs.argmax(dim=1)
        
        # 请将 ___________ 替换为计算准确率的代码
        accuracy = ___________  # 期望：返回 0-1 之间的准确率
    
    return accuracy

# 评估两个模型
print("="*60)
print("📊 模型性能评估")
print("="*60)

# 普通模型
normal_train_acc = evaluate_model(normal_model, X_train, y_train)
normal_test_acc = evaluate_model(normal_model, X_test, y_test)

# DP 模型
dp_train_acc = evaluate_model(dp_model, X_train, y_train)
dp_test_acc = evaluate_model(dp_model, X_test, y_test)

print(f"\n  {'模型类型':<15} {'训练准确率':<15} {'测试准确率':<15}")
print("  " + "-"*45)
print(f"  {'普通模型':<15} {normal_train_acc:>13.2%} {normal_test_acc:>13.2%}")
print(f"  {'DP模型 (σ=1.0)':<15} {dp_train_acc:>13.2%} {dp_test_acc:>13.2%}")
print("\n  📉 性能下降：")
print(f"     训练集: {(normal_train_acc - dp_train_acc):.2%}")
print(f"     测试集: {(normal_test_acc - dp_test_acc):.2%}")

if (normal_test_acc - dp_test_acc) < 0.15:
    print("\n  ✓ 性能下降在可接受范围内（<15%）")
else:
    print("\n  ⚠ 性能下降较大，可能需要调整隐私参数")

### 4.2 测试对成员推理攻击的防御效果

In [ ]:
# ========== 填空 4：实现成员推理攻击 ==========
# 
# 🎯 任务：基于模型置信度执行成员推理攻击，评估隐私泄露风险
# 
# 💡 提示：
#   - 成员样本通常有更高的预测置信度
#   - 使用 torch.softmax() 将 logits 转换为概率
#   - gather() 函数可以提取正确类别的概率
#   - 使用阈值判断：置信度 > 阈值 → 判定为成员
#
# 难度：⭐⭐⭐☆☆

def membership_inference_attack(model, X_train, y_train, X_test, y_test):
    """
    执行基于置信度的成员推理攻击
    
    参数:
        model: 目标模型
        X_train, y_train: 训练集（成员样本）
        X_test, y_test: 测试集（非成员样本）
    
    返回:
        attack_acc: 攻击准确率
        train_conf: 训练样本的置信度数组
        test_conf: 测试样本的置信度数组
    """
    model.eval()
    
    with torch.no_grad():
        # 步骤1：获取模型在两个数据集上的输出概率
        # 请将 ___________ 替换为计算 softmax 概率的代码
        train_probs = ___________  # 期望：形状 [n_train, n_classes]
        test_probs = ___________   # 期望：形状 [n_test, n_classes]
        
        # 步骤2：提取正确类别的置信度（真实标签对应的概率）
        train_conf = train_probs.gather(1, y_train.unsqueeze(1)).squeeze().cpu().numpy()
        test_conf = test_probs.gather(1, y_test.unsqueeze(1)).squeeze().cpu().numpy()
    
    # 步骤3：设置攻击阈值（使用两组数据的中位数置信度的平均值）
    threshold = (np.median(train_conf) + np.median(test_conf)) / 2
    
    # 步骤4：执行攻击
    # 规则：如果置信度 > 阈值，判定为成员；否则判定为非成员
    member_correct = np.mean(train_conf > threshold)      # 成员样本被正确识别的比例
    non_member_correct = np.mean(test_conf <= threshold)  # 非成员样本被正确识别的比例
    
    # 攻击总准确率
    attack_acc = (member_correct + non_member_correct) / 2
    
    return attack_acc, train_conf, test_conf, threshold

# 对两个模型执行成员推理攻击
print("="*60)
print("🎯 成员推理攻击评估")
print("="*60)

normal_attack_acc, normal_train_conf, normal_test_conf, normal_threshold = \
    membership_inference_attack(normal_model, X_train, y_train, X_test, y_test)

dp_attack_acc, dp_train_conf, dp_test_conf, dp_threshold = \
    membership_inference_attack(dp_model, X_train, y_train, X_test, y_test)

print(f"\n  {'模型类型':<15} {'攻击准确率':<15} {'判断阈值':<15}")
print("  " + "-"*45)
print(f"  {'普通模型':<15} {normal_attack_acc:>13.2%} {normal_threshold:>13.4f}")
print(f"  {'DP模型 (σ=1.0)':<15} {dp_attack_acc:>13.2%} {dp_threshold:>13.4f}")

defense_improvement = normal_attack_acc - dp_attack_acc
print(f"\n  🛡️ 防御效果: 攻击准确率降低 {defense_improvement:.2%}")

if dp_attack_acc < 0.55:
    print("  ✓ 差分隐私显著降低了隐私泄露风险！")
elif dp_attack_acc < 0.65:
    print("  ⚠ 差分隐私提供了一定保护，但仍有改进空间")
else:
    print("  ✗ 当前隐私参数可能不足，建议增加噪声强度")

print(f"\n  💡 参考：随机猜测的攻击准确率为 50%")

### 4.3 可视化成员与非成员的置信度分布

In [ ]:
# 绘制置信度分布对比图
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 普通模型的置信度分布
axes[0].hist(normal_train_conf, bins=30, alpha=0.6, label='成员（训练集）', 
             color='#FF6B6B', edgecolor='black')
axes[0].hist(normal_test_conf, bins=30, alpha=0.6, label='非成员（测试集）', 
             color='#4ECDC4', edgecolor='black')
axes[0].axvline(normal_threshold, color='red', linestyle='--', linewidth=2, 
                label=f'攻击阈值={normal_threshold:.3f}')
axes[0].set_xlabel('预测置信度', fontsize=12)
axes[0].set_ylabel('样本数量', fontsize=12)
axes[0].set_title(f'普通模型\n攻击准确率: {normal_attack_acc:.1%}', 
                  fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# DP 模型的置信度分布
axes[1].hist(dp_train_conf, bins=30, alpha=0.6, label='成员（训练集）', 
             color='#FF6B6B', edgecolor='black')
axes[1].hist(dp_test_conf, bins=30, alpha=0.6, label='非成员（测试集）', 
             color='#4ECDC4', edgecolor='black')
axes[1].axvline(dp_threshold, color='red', linestyle='--', linewidth=2, 
                label=f'攻击阈值={dp_threshold:.3f}')
axes[1].set_xlabel('预测置信度', fontsize=12)
axes[1].set_ylabel('样本数量', fontsize=12)
axes[1].set_title(f'差分隐私模型 (σ=1.0)\n攻击准确率: {dp_attack_acc:.1%}', 
                  fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')

plt.suptitle('🔍 成员 vs 非成员置信度分布对比', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n📊 分析：")
print("  - 普通模型：成员和非成员的置信度分布差异明显，易于区分")
print("  - DP 模型：两个分布更加重叠，攻击者难以判断样本是否为成员")
print("  - 差分隐私通过噪声模糊了成员和非成员的边界，提升了隐私保护")

### 🤔 思考一下

1. **为什么成员样本通常有更高的置信度？** 这与模型的什么特性有关？

2. **如果攻击准确率接近 50%，说明什么？** 这对隐私保护意味着什么？

## 第五部分：隐私-效用权衡分析

### 5.1 探索不同噪声强度的影响

In [ ]:
# ========== 填空 5：隐私-效用权衡实验 ==========
# 
# 🎯 任务：测试不同噪声强度下的模型性能和隐私保护效果
# 
# 💡 提示：
#   - noise_multiplier=0 相当于普通训练（无隐私保护）
#   - noise_multiplier 越大，隐私保护越强，但模型性能越差
#   - 需要在循环中训练多个模型，收集性能和攻击准确率数据
#
# 难度：⭐⭐⭐⭐☆

print("="*60)
print("⚖️ 隐私-效用权衡分析")
print("="*60)
print("\n测试不同噪声强度的影响...\n")

# 定义要测试的噪声强度范围
noise_multipliers = [0.0, 0.5, 1.0, 1.5, 2.0, 3.0]
results = []

for i, noise_mult in enumerate(noise_multipliers):
    print(f"[{i+1}/{len(noise_multipliers)}] 训练模型 (噪声强度={noise_mult})...")
    
    # 创建新模型
    temp_model = SimpleClassifier()
    
    # 根据噪声强度选择训练方式
    if noise_mult == 0:
        # noise_mult=0 表示普通训练
        # 请将 ___________ 替换为调用普通训练函数的代码
        ___________
    else:
        # noise_mult>0 表示差分隐私训练
        # 请将 ___________ 替换为调用 DP 训练函数的代码
        ___________
    
    # 评估模型性能
    test_acc = evaluate_model(temp_model, X_test, y_test)
    
    # 评估隐私泄露风险（成员推理攻击）
    attack_acc, _, _, _ = membership_inference_attack(
        temp_model, X_train, y_train, X_test, y_test
    )
    
    # 记录结果
    results.append({
        'noise': noise_mult,
        'test_acc': test_acc,
        'attack_acc': attack_acc,
        'privacy_gain': normal_attack_acc - attack_acc,  # 相对普通模型的隐私提升
        'utility_loss': normal_test_acc - test_acc        # 相对普通模型的性能损失
    })
    
    print(f"  测试准确率: {test_acc:.2%} | 攻击准确率: {attack_acc:.2%}\n")

print("✓ 实验完成！\n")

### 5.2 可视化隐私-效用权衡曲线

In [ ]:
# 提取数据
noise_list = [r['noise'] for r in results]
test_acc_list = [r['test_acc'] for r in results]
attack_acc_list = [r['attack_acc'] for r in results]

# 创建双 Y 轴图表
fig, ax1 = plt.subplots(figsize=(12, 6))

# 左侧 Y 轴：模型测试准确率（效用）
color1 = '#4ECDC4'
ax1.set_xlabel('噪声强度 (noise_multiplier σ)', fontsize=13, fontweight='bold')
ax1.set_ylabel('模型测试准确率（效用）', color=color1, fontsize=13, fontweight='bold')
line1 = ax1.plot(noise_list, test_acc_list, 'o-', color=color1, linewidth=3, 
                 markersize=10, label='测试准确率')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.set_ylim([0.4, 1.0])
ax1.grid(True, alpha=0.3, linestyle='--')

# 右侧 Y 轴：成员推理攻击准确率（隐私风险）
ax2 = ax1.twinx()
color2 = '#FF6B6B'
ax2.set_ylabel('成员推理攻击准确率（隐私风险）', color=color2, fontsize=13, fontweight='bold')
line2 = ax2.plot(noise_list, attack_acc_list, 's--', color=color2, linewidth=3, 
                 markersize=10, label='攻击准确率')
ax2.tick_params(axis='y', labelcolor=color2)
ax2.set_ylim([0.4, 1.0])

# 添加随机猜测基准线
ax2.axhline(y=0.5, color='gray', linestyle=':', linewidth=2, label='随机猜测 (50%)')

# 添加图例
lines = line1 + line2
labels = [l.get_label() for l in lines] + ['随机猜测 (50%)']
ax1.legend(lines + [ax2.lines[-1]], labels, loc='center left', fontsize=11, 
          bbox_to_anchor=(0.02, 0.5))

plt.title('⚖️ 隐私-效用权衡：噪声强度的影响', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📈 关键观察：")
print("  1. 噪声强度 ↑ → 模型效用 ↓（蓝色曲线下降）")
print("  2. 噪声强度 ↑ → 隐私风险 ↓（红色曲线下降，趋近 50%）")
print("  3. 存在一个最佳平衡点，在该点隐私保护充分且性能损失可控")

### 5.3 生成详细的权衡分析表格

In [ ]:
# 打印详细结果表格
print("="*80)
print("📊 隐私-效用权衡详细分析表")
print("="*80)
print(f"\n  {'噪声强度':<12} {'测试准确率':<12} {'攻击准确率':<12} {'隐私提升':<12} {'性能损失':<12}")
print("  " + "-"*68)

for r in results:
    noise_str = f"σ={r['noise']:.1f}"
    if r['noise'] == 0:
        noise_str += " (普通)"
    
    print(f"  {noise_str:<12} {r['test_acc']:>10.2%} {r['attack_acc']:>10.2%} "
          f"{r['privacy_gain']:>10.2%} {r['utility_loss']:>10.2%}")

print("\n  💡 说明：")
print("     - 隐私提升 = 相比普通模型，攻击准确率下降的幅度")
print("     - 性能损失 = 相比普通模型，测试准确率下降的幅度")

# 找到推荐的噪声强度
# 策略：隐私提升 > 10% 且性能损失 < 15% 的最大噪声强度
recommended = None
for r in results:
    if r['privacy_gain'] > 0.10 and r['utility_loss'] < 0.15 and r['noise'] > 0:
        recommended = r

if recommended:
    print(f"\n  🎯 推荐配置: σ={recommended['noise']:.1f}")
    print(f"     - 隐私提升: {recommended['privacy_gain']:.2%}")
    print(f"     - 性能损失: {recommended['utility_loss']:.2%}")
    print(f"     - 该配置在隐私保护和模型效用之间取得了良好平衡")
else:
    print("\n  ⚠ 提示: 可能需要进一步调整参数以找到更好的平衡点")

运行到这里，你应该看到：
- 成功训练了 6 个不同噪声强度的模型
- 测试准确率随噪声强度增加而下降
- 攻击准确率随噪声强度增加而下降（趋近 50%）
- 权衡曲线清晰展示了隐私与效用的反向关系

如果发现某些配置下模型完全失效（准确率 < 40%），说明噪声强度过大，可以尝试增加训练轮数或调整学习率。

## 📋 实验小结

### 核心收获

通过本实验，你已经：

| 目标 | 完成情况 |
|------|----------|
| 理解差分隐私核心机制 | ✅ 掌握梯度裁剪与噪声添加 |
| 实现 DP-SGD 训练 | ✅ 完成简化版差分隐私训练流程 |
| 评估隐私保护效果 | ✅ 成员推理攻击准确率显著下降 |
| 分析隐私-效用权衡 | ✅ 理解 ε 与模型性能的关系 |

### 🔑 关键代码回顾

**差分隐私 SGD 的核心代码：**

```python
# 1. 梯度裁剪（限制单个样本的影响）
def clip_gradient(gradient, max_norm):
    grad_norm = torch.norm(gradient)
    if grad_norm > max_norm:
        gradient = gradient * (max_norm / grad_norm)
    return gradient

# 2. 噪声添加（保护样本隐私）
def add_noise(avg_gradient, max_norm, noise_multiplier, n_samples):
    noise_std = (max_norm * noise_multiplier) / n_samples
    noise = torch.randn_like(avg_gradient) * noise_std
    return avg_gradient + noise
```

### 🤔 思考与讨论

1. **差分隐私能防御所有隐私攻击吗？**
   - DP 主要防御成员推理攻击，但对训练数据提取攻击的效果有限
   - 需要结合其他防御措施（如数据去标识化、访问控制等）

2. **如何在实际项目中选择隐私参数？**
   - 高敏感场景（医疗、金融）：ε ≤ 1
   - 中等敏感场景（推荐系统）：ε ∈ [1, 3]
   - 低敏感场景（公开数据集）：ε ∈ [3, 10]

### 🔗 延伸阅读

- **经典论文**：Abadi et al. (2016) - "Deep Learning with Differential Privacy"
- **实用工具**：[Opacus](https://opacus.ai/) - PyTorch 的差分隐私库
- **理论基础**：Dwork & Roth (2014) - "The Algorithmic Foundations of Differential Privacy"

## 📖 参考答案

<details>
<summary>点击展开参考答案</summary>

**填空 1：梯度裁剪函数**
```python
if grad_norm > max_norm:
    gradient = gradient * (max_norm / grad_norm)
```

**填空 2：噪声添加机制**
```python
noise = torch.randn_like(avg_gradient) * noise_std
```

**填空 3：模型评估函数**
```python
accuracy = (predictions == y).float().mean().item()
```

**填空 4：成员推理攻击**
```python
train_probs = torch.softmax(model(X_train), dim=1)
test_probs = torch.softmax(model(X_test), dim=1)
```

**填空 5：隐私-效用权衡实验**
```python
# 普通训练
train_normal(temp_model, X_train, y_train, epochs=50, lr=0.01, verbose=False)

# 差分隐私训练
train_with_dp(temp_model, X_train, y_train, epochs=50, lr=0.01,
             max_grad_norm=1.0, noise_multiplier=noise_mult, verbose=False)
```

</details>

⚠️ **重要提醒**

实验完成后，请记得：

1. **释放 GPU 资源**：如果你使用的是 Tencent CloudStudio，请点击右上角的「停止」按钮，停止运行。
2. **保存实验结果**：如需保存，请下载 notebook 文件

这样可以避免不必要的资源占用和费用产生。**否则会一直消耗你的免费资源额度。**